# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
# def calculate_mean_win_perc_entry_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     entry_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values
        
#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)

#             commission = exec_dict['COMMISSION']
#             slippage = exec_dict['SLIPPAGE'] 
#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE'] 

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)
#             signal_idxs_true = [i - 1 for i in signal_idxs]

#             buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
#             df['buy'] = 0
#             df.loc[df.index.isin(buy_idxs), 'buy'] = 1

#             sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
#             df['sell'] = 0
#             df.loc[df.index.isin(sell_idxs), 'sell'] = -1

#             df['signal'] = df['buy'] + df['sell']

#             df['new_signal'] = 0
#             df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
#             df['signal_prices'] = 0
#             df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

#             signal_list = np.zeros((2, df.shape[0]))
#             signal_list[0][1:] = df['new_signal'].values[:-1]
#             signal_list[1] = df['signal_prices'].values

#             # fixed stop and target exit testing
#             exit_list = get_exit_entry_testing1(
#                 close_prices=df['btc_close'].values, 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list,
#                 stoploss_th=50,
#                 takeprofit_th=100,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)

#             # fixed bar exit testing
#             exit_list = get_exit_entry_testing2( 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list,
#                 n_exit_bars=5
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)

#             # random exit testing
#             exit_list = get_exit_entry_testing3( 
#                 open_prices=df['btc_open'].values,  
#                 signal_list=signal_list
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)

#         except:

#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)

#     mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
#         append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
#     mean_win_perc_dict['Fixed_Bar_testing'].\
#         append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
#     mean_win_perc_dict['Random_Exit_testing'].\
#         append(np.mean(entry_walk_forward_dict['random_exit_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [7]:
def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
    df['signal_prices'] = 0
    df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df


## Exit testing

### Similar approach entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [9]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [10]:
# def calculate_mean_win_perc_exit_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     exit_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:

#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)

#             commission = exec_dict['COMMISSION']
#             slippage = exec_dict['SLIPPAGE'] 
#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)
#             signal_idxs_true = [i - 1 for i in signal_idxs]

#             buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
#             df['buy'] = 0
#             df.loc[df.index.isin(buy_idxs), 'buy'] = 1

#             sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
#             df['sell'] = 0
#             df.loc[df.index.isin(sell_idxs), 'sell'] = -1

#             df['signal'] = df['buy'] + df['sell']

#             df['exit_signal'] = 0
#             df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
#             df['exit_prices'] = 0
#             df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

#             exit_list = np.zeros((2, df.shape[0]))
#             exit_list[0][1:] = df['exit_signal'].values[:-1]
#             exit_list[1] = df['exit_prices'].values

#             # replacing entry with trend following entry
#             signal_list = get_entry_exit_testing1(
#                 close_prices=df['btc_close'].values,
#                 open_prices=df['btc_open'].values,
#                 n_bars=5
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['trend_entry_testing'].append(fixed_winning_percent)

#             # replacing entry with countertrend entry
#             signal_list = get_entry_exit_testing2(
#                 close_prices=df['btc_close'].values, 
#                 open_prices=df['btc_open'].values, 
#                 prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
#                 rsi_window_size=10, 
#                 rsi_threshold = 20
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['countertrend_entry_testing'].append(fixed_bar_winning_percent)

#             # random entry testing
#             signal_list = get_entry_exit_testing3(
#                 open_prices=df['btc_open'].values
#             )

#             signal_list, exit_list = get_signals(signal_list, exit_list)
#             pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

#             pnl_list = get_pnl_testing(
#                 trade_close_prices=pos_exit_prices,
#                 signal_list=exit_list[0], 
#                 trade_open_prices=pos_open_prices,
#                 commission=commission, 
#                 slippage=slippage, 
#                 init_inv=init_inv, 
#                 trade_size=trade_size
#             )

#             random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)

#         except:

#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)

#     mean_win_perc_dict['Trend_testing'].\
#         append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
#     mean_win_perc_dict['Countertrend_testing'].\
#         append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
#     mean_win_perc_dict['Random_Entry_testing'].\
#         append(np.mean(exit_walk_forward_dict['random_entry_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [11]:
def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)
    signal_idxs_true = [i - 1 for i in signal_idxs]

    buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
    df['buy'] = 0
    df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
    df['sell'] = 0
    df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
    df['exit_prices'] = 0
    df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df


## Core testing

### Winning percentage

In [12]:
# def calculate_mean_win_perc_core_testing(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     core_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
#             core_walk_forward_dict['core_testing'].append(winning_percent)
#         except:
#             n_not_worked += 1

#     mean_win_perc_dict = defaultdict(list)
#     mean_win_perc_dict['Core_Testing'].\
#         append(np.mean(core_walk_forward_dict['core_testing']))
#     mean_win_perc_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     win_pc_df = pd.DataFrame(mean_win_perc_dict)

#     return win_pc_df


In [13]:
def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df


### Performance

In [14]:
# @njit(cache=True)
# def get_drawdown(pnl_list):

#     # running_total = np.cumsum(pnl_list)
#     # max_running_total = np.maximum.accumulate(running_total)

#     running_total = np.zeros(len(pnl_list))
#     max_running_total = np.zeros(len(pnl_list))

#     for i in range(len(pnl_list)):
#         if i == 0:
#             running_total[i] = pnl_list[i]
#             max_running_total[i] = pnl_list[i]
#         else:
#             running_total[i] = running_total[i-1] + pnl_list[i]
#             max_running_total[i] = max([max_running_total[i-1], pnl_list[i]])

#     min_total = running_total[np.argmin(running_total)]

#     max_total = max_running_total[np.argmin(running_total)]

#     if max_total == 0:
#         max_total = 0.00001

#     max_drawdown = 100 * (min_total - max_total) / max_total

#     if max_drawdown == -1:
#         max_drawdown = 0

#     return max_drawdown

@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio



In [15]:
# def calculate_mean_performance(data, text_code, monkey_test=False):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     performance_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             signal_idxs = list(exec_dict['buy_idxs'])
#             signal_idxs.extend(list(exec_dict['sell_idxs']))
#             signal_idxs = sorted(signal_idxs)

#             performance_walk_forward_dict['n_trades'].append(len(signal_idxs) - 1)
#             overall_pnl = np.sum(pnl_list)
#             performance_walk_forward_dict['pnl'].append(overall_pnl)
#             performance_walk_forward_dict['roi'].append(100 * overall_pnl / (trade_size * init_inv))
#             performance_walk_forward_dict['avg_drawdown'].append(exec_dict['avg_drawdown'])
#             max_dd = get_drawdown(pnl_list)
#             performance_walk_forward_dict['drawdown'].append(max_dd)
#             performance_walk_forward_dict['drawdown_dur'].append(get_drawdown_duration(pnl_list))
#             performance_walk_forward_dict['pnl_avgd_ratio'].append(exec_dict['fitness'])
#             annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
#             performance_walk_forward_dict['sharpe_ratio'].append(annualized_sharpe_ration)
#             annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
#             performance_walk_forward_dict['sortino_ratio'].append(annualized_sortino_ratio)

#             if monkey_test:
#                 pnl_mren_arr = exec_dict['pnl_mren_arr']
#                 max_dd_mren_arr = exec_dict['max_dd_mren_arr']

#                 pnl_mren_good_cases = np.sum(np.where(overall_pnl > pnl_mren_arr, 1, 0))
#                 max_dd_mren_good_cases = np.sum(np.where(max_dd < max_dd_mren_arr, 1, 0))
#                 performance_walk_forward_dict['mt_pnl'].append(100 * pnl_mren_good_cases / len(pnl_mren_arr))
#                 performance_walk_forward_dict['mt_mdd'].append(100 * max_dd_mren_good_cases / len(max_dd_mren_arr))

#         except:
#             n_not_worked += 1

#     mean_perf_dict = defaultdict(list)
#     mean_perf_dict['N_Trades'].\
#         append(np.mean(performance_walk_forward_dict['n_trades']))
#     mean_perf_dict['PNL'].\
#         append(np.mean(performance_walk_forward_dict['pnl']))
#     mean_perf_dict['ROI (%)'].\
#         append(np.mean(performance_walk_forward_dict['roi']))
#     mean_perf_dict['AVG_Drawdown'].\
#         append(np.mean(performance_walk_forward_dict['avg_drawdown']))
#     mean_perf_dict['Drawdown (%)'].\
#         append(np.mean(performance_walk_forward_dict['drawdown']))
#     mean_perf_dict['Drawdown_Duration'].\
#         append(np.mean(performance_walk_forward_dict['drawdown_dur']))
#     mean_perf_dict['PNL_AVGD_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
#     mean_perf_dict['Sharpe_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
#     mean_perf_dict['Sortino_Ratio'].\
#         append(np.mean(performance_walk_forward_dict['sortino_ratio']))
#     mean_perf_dict['Not_Working'].\
#         append(100 * n_not_worked / n_total_cases)
    
#     if monkey_test:
#         mean_perf_dict['MT_PNL_D'].\
#             append(np.mean(performance_walk_forward_dict['mt_pnl']))
#         mean_perf_dict['MT_MDD_D'].\
#             append(np.mean(performance_walk_forward_dict['mt_mdd']))
#         # mean_perf_dict['MT_PNL_D_Min'].\
#         #     append(np.min(performance_walk_forward_dict['mt_pnl']))
#         # mean_perf_dict['MT_MDD_D_Min'].\
#         #     append(np.min(performance_walk_forward_dict['mt_mdd']))
#         # mean_perf_dict['MT_PNL_D_Max'].\
#         #     append(np.max(performance_walk_forward_dict['mt_pnl']))
#         # mean_perf_dict['MT_MDD_D_Max'].\
#         #     append(np.max(performance_walk_forward_dict['mt_mdd']))
    
#     perf_df = pd.DataFrame(mean_perf_dict)

#     return perf_df


In [16]:
def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.mean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.mean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.mean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


## Equity curve Monte Carlo simulation

In [17]:
@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array


In [18]:
def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

In [19]:
# def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

#     max_dd_list = []
#     dd_dur_list = []
#     profit_list = []
#     roi_list = []
#     binary_profit_list = []

#     for _ in range(n_runs):

#         temp_pnl_list = np.random.choice(pnl_list, size=len(pnl_list), replace=True)

#         max_dd = get_drawdown(temp_pnl_list)
#         max_dd_list.append(max_dd)

#         dd_dur = get_drawdown_duration(temp_pnl_list)
#         dd_dur_list.append(dd_dur)

#         profit_list.append(np.sum(temp_pnl_list))
#         roi_list.append(100 * np.sum(temp_pnl_list) / (init_inv * trade_size))
#         is_profit = 1 if np.sum(temp_pnl_list) > 0 else 0
#         binary_profit_list.append(is_profit)

#     mc_dict = {}
#     mc_dict['median_max_dd'] = np.median(max_dd_list)
#     mc_dict['median_dd_dur'] = np.median(dd_dur_list)
#     mc_dict['median_profit'] = np.median(profit_list)
#     mc_dict['median_return'] = np.median(roi_list)
#     mc_dict['return_dd_ratio'] = np.median(roi_list) / np.median(max_dd_list)
#     mc_dict['prob_profit'] = np.sum(binary_profit_list) / len(binary_profit_list)

#     return mc_dict

In [20]:
# def calculate_mc_performance(data, text_code):

#     bars_per_5week = 7 * 60 * 24 * 5
#     n_bars_per_year = 7 * 60 * 24 * 52

#     if data.shape[0] < n_bars_per_year:
#         n_bars_per_year = data.shape[0]

#     n_not_worked = 0
#     n_total_cases = 0

#     performance_walk_forward_dict = defaultdict(list)

#     for idx in range(0, n_bars_per_year, bars_per_5week):

#         n_total_cases += 1

#         df = data.iloc[idx:idx+bars_per_5week, :]
#         df.reset_index(drop=True, inplace=True)

#         price_data = {}
#         for col in df.columns:
#             if col == 'datetime':
#                 continue
#             else:
#                 price_data[col] = df[col].values

#         try:
#             exec_dict = {'price_data': price_data}
#             exec(text_code, exec_dict)
#             pnl_list = exec_dict['all_arr']

#             init_inv = exec_dict['AVAILABLE_CAPITAL']
#             trade_size = exec_dict['TRADE_SIZE']       

#             mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

#             performance_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
#             performance_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
#             performance_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
#             performance_walk_forward_dict['median_return'].append(mc_dict['median_return'])
#             performance_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
#             performance_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
#         except:
#             n_not_worked += 1

#     mean_perf_dict = defaultdict(list)
#     mean_perf_dict['median_drawdown (%)'].\
#         append(np.mean(performance_walk_forward_dict['median_max_dd']))
#     mean_perf_dict['median_drawdown_duration'].\
#         append(np.mean(performance_walk_forward_dict['median_dd_dur']))
#     mean_perf_dict['median_profit'].\
#         append(np.mean(performance_walk_forward_dict['median_profit']))
#     mean_perf_dict['median_ROI (%)'].\
#         append(np.mean(performance_walk_forward_dict['median_return']))
#     mean_perf_dict['ratio'].\
#         append(np.mean(performance_walk_forward_dict['return_dd_ratio']))
#     mean_perf_dict['prob'].\
#         append(np.mean(performance_walk_forward_dict['prob_profit']))
    
#     perf_df = pd.DataFrame(mean_perf_dict)

#     return perf_df


In [21]:
def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df


# Data loading function

In [22]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.iloc[-(7 * 60 * 24 * 52):]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [ ]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min.csv')
df_52w = generate_52w_data(data_path)
df_52w.head()

,datetime,6e_open,6e_high,6e_low,6e_close,6e_volume,aapl_open,aapl_high,aapl_low,aapl_close,...,zf_open,zf_high,zf_low,zf_close,zf_volume,zn_open,zn_high,zn_low,zn_close,zn_volume
0,2023-11-01 18:56:00,1.05600,1.05635,1.05590,1.05590,607,172.77,172.97,172.765,172.95,...,104.914062,104.945312,104.914062,104.937500,3727,106.812500,106.859375,106.796875,106.843750,11709
1,2023-11-01 18:58:00,1.05625,1.05640,1.05615,1.05630,396,173.10,173.27,173.060,173.25,...,104.937500,104.945312,104.921875,104.937500,3415,106.843750,106.859375,106.828125,106.859375,8757
2,2023-11-01 18:59:00,1.05630,1.05650,1.05605,1.05615,1569,173.24,173.24,173.090,173.19,...,104.937500,104.960938,104.929688,104.960938,13873,106.859375,106.875000,106.828125,106.875000,33414
3,2023-11-01 19:02:00,1.05650,1.05670,1.05640,1.05660,326,173.59,173.75,173.550,173.74,...,104.976562,105.015625,104.968750,105.007812,5523,106.906250,106.937500,106.890625,106.921875,7998
4,2023-11-01 19:03:00,1.05655,1.05675,1.05645,1.05665,327,173.72,173.73,173.650,173.72,...,105.000000,105.023438,105.000000,105.000000,4571,106.921875,106.953125,106.906250,106.906250,9335


## Loading strategies generated

In [24]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\derived_strategies/\ge_results_inst_ind_comb_numba.csv')
strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (2).csv")
strategy_file_path = Path(r"C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_10_30_180400_225305_2676_225305/\ge_results.csv")

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1.5)]
df_str = df_str[~df_str.duplicated()]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str

,buy,sell,fitness
0,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872
1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872
2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872
3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872
4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-61876.559872
...,...,...,...
38117,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-1.514911
38118,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-1.512986
38119,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-1.507192
38120,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,-1.506468


In [25]:
# buy_list = [
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_max(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_min(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] < 30)",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] > 70)",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",

# ]

# sell_list = [
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_min(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] == signals.moving_max(price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:])",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] > 70)",
#     "(numba_indicators.relative_strength_index(prices=price_data["'"btc_close"'"], window=<window-const>)[MAX_LAG:] < 30)",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > get_lag(price_data["'"btc_close"'"], lag=<lag-steps>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] < numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",
#     "(price_data["'"btc_close"'"][MAX_LAG:] > numba_indicators.moving_average(prices=price_data["'"btc_close"'"], window=<short-window-const>)[MAX_LAG:])",
# ]

# df_str = pd.DataFrame()
# df_str['buy'] = buy_list
# df_str['sell'] = sell_list
# df_str

## Running the tests over the strategies

In [26]:
# final_entry_win_pc_df = pd.DataFrame()
# final_exit_win_pc_df = pd.DataFrame()
# final_core_win_pc_df = pd.DataFrame()
# final_perf_df = pd.DataFrame()
# final_mc_df = pd.DataFrame()

# lag_txt = '{i}'

# strategy_idx = 1

# for row in tqdm(df_str.iloc[:3000].itertuples()):#df_str.iloc[:10].itertuples()

#     buy_signal_txt = row.buy
#     sell_signal_txt = row.sell

#     txt_code = f'''import os
# CUR_DIR = os.getcwd()
# os.chdir('src')
# #import pandas as pd
# import numpy as np
# import gc
# from fitness.indicators import numba_indicators
# from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag, get_monkey_test_results
# os.chdir(CUR_DIR)
# #from numba import njit
# COMMISSION = 0.015
# SLIPPAGE = 0.00005
# AVAILABLE_CAPITAL = 700000
# TRADE_SIZE = 0.5
# MAX_LAG = 5
# buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
# if len(buy_idxs) == 0 or len(sell_idxs) == 0:
#     fitness = -999
# else:
#     buy_idxs = np.array(buy_idxs)
#     sell_idxs = np.array(sell_idxs)
#     open_prices = price_data['btc_open']
#     pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
#     buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
#     sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
#     if buy_idxs[0] < sell_idxs[0]:
#         buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
#         buy_pnl = np.sum(buy_arr)
#         sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
#         sell_pnl = np.sum(sell_arr)
#         all_arr = merge_pnl(buy_arr, sell_arr)
#     else:
#         sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
#         sell_pnl = np.sum(sell_arr)
#         buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
#         buy_pnl = np.sum(buy_arr)
#         all_arr = merge_pnl(sell_arr, buy_arr)
#     total_pnl = buy_pnl + sell_pnl
#     equity_curve_arr = np.cumsum(all_arr)
#     drawdowns = get_drawdowns(equity_curve_arr)
#     avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
#     fitness = total_pnl / avg_drawdown
#     if np.isnan(fitness) or total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
#         fitness = -999
# gc.collect()'''

#     entry_win_pc_df = calculate_mean_win_perc_entry_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})
#     entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
#     final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

#     exit_win_pc_df = calculate_mean_win_perc_exit_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
#     final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

#     core_win_pc_df = calculate_mean_win_perc_core_testing(
#         data=df_52w, 
#         text_code=txt_code
#     )

#     core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
#     final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

#     perf_df = calculate_mean_performance(
#         data=df_52w, 
#         text_code=txt_code,
#         monkey_test=False
#     )

#     perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
#     final_perf_df = pd.concat([final_perf_df, perf_df])

#     # mc_df = calculate_mc_performance(
#     #     data=df_52w, 
#     #     text_code=txt_code
#     # )

#     # mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
#     # final_mc_df = pd.concat([final_mc_df, mc_df])

#     strategy_idx += 1

In [ ]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = {}

bars_per_5week = 7 * 60 * 24 * 5
n_bars_per_year = 7 * 60 * 24 * 52

if df_52w.shape[0] < n_bars_per_year:
    n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    sell_signal_txt = row.sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_pnl, get_drawdowns, get_pnl, trading_signals, get_lag, get_monkey_test_results
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, sell_idxs = trading_signals(buy_signal={buy_signal_txt}, sell_signal={sell_signal_txt})
if len(buy_idxs) == 0 or len(sell_idxs) == 0:
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    if buy_idxs[0] < sell_idxs[0]:
        buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        all_arr = merge_pnl(buy_arr, sell_arr)
    else:
        sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
        sell_pnl = np.sum(sell_arr)
        buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
        buy_pnl = np.sum(buy_arr)
        all_arr = merge_pnl(sell_arr, buy_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    elif total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    equity_curve_dict = defaultdict(list)

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = df_52w.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        try:
            equity_curve_arr = exec_dict['equity_curve_arr']
            equity_curve_dict[strategy_idx].append(equity_curve_arr)
        except:
            pass

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=True)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

31076it [9:16:59,  1.08s/it] 


In [32]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True) 
final_perf_df.reset_index(drop=True, inplace=True)
final_mc_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [33]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,48.192697,15.626409,19.645528,0.0
1,strategy2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,30.086664,16.086467,32.885947,0.0
2,strategy3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,26.257479,16.088845,13.705001,0.0
3,strategy4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,30.086664,16.086467,17.060077,0.0
4,strategy5,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,30.086664,16.086467,44.918866,0.0
...,...,...,...,...,...,...,...
38117,strategy38118,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,58.472808,35.702885,39.544543,0.0
38118,strategy38119,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,67.933871,59.249028,47.668447,0.0
38119,strategy38120,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,46.687400,65.070595,65.207072,0.0
38120,strategy38121,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,51.978557,47.084101,55.001023,0.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [34]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
267,strategy268,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,60.045905,70.417282,65.137013,0.0
340,strategy341,((price_data['btc_close'][MAX_LAG:] < numba_in...,(numba_indicators.relative_strength_index(pric...,64.043036,60.648461,68.133936,0.0
460,strategy461,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,61.564166,91.178038,65.163287,0.0
461,strategy462,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,62.429049,76.129680,69.869470,0.0
462,strategy463,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,62.429049,76.129680,65.736358,0.0
...,...,...,...,...,...,...,...
37952,strategy37953,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,68.625445,81.137712,65.480551,0.0
37980,strategy37981,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((numba_indicators.relative_strength_index(pri...,63.834793,71.178396,64.839211,0.0
37996,strategy37997,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,68.082800,80.274113,67.995582,0.0
37997,strategy37998,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,68.191209,64.207835,77.295506,0.0


## Showing exit testing results

In [35]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.356361,60.428475,49.801579,0.0
1,strategy2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.374772,60.426429,50.320152,0.0
2,strategy3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.368635,60.426429,48.660038,0.0
3,strategy4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.374772,60.426429,49.433352,0.0
4,strategy5,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.374772,60.426429,48.020045,0.0
...,...,...,...,...,...,...,...
38117,strategy38118,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.321584,60.526693,49.517224,0.0
38118,strategy38119,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.288853,60.569663,48.468918,0.0
38119,strategy38120,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.315447,60.538970,50.558396,0.0
38120,strategy38121,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.202934,60.405967,50.047048,0.0


### Showing strategies that have passed entry testing

In [36]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
267,strategy268,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,59.994272,60.172700,48.249131,0.0
340,strategy341,((price_data['btc_close'][MAX_LAG:] < numba_in...,(numba_indicators.relative_strength_index(pric...,60.119060,60.352765,48.005727,0.0
460,strategy461,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,59.689463,60.802930,50.104333,0.0
461,strategy462,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,59.865393,60.178838,48.695866,0.0
462,strategy463,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,59.865393,60.178838,49.242099,0.0
...,...,...,...,...,...,...,...
37952,strategy37953,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.335904,60.764052,49.303467,0.0
37980,strategy37981,((price_data['btc_close'][MAX_LAG:] < get_lag(...,((numba_indicators.relative_strength_index(pri...,60.266350,60.414152,50.378447,0.0
37996,strategy37997,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.352270,60.653557,50.964469,0.0
37997,strategy37998,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,60.352270,60.653557,49.295314,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [37]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
4880,strategy4881,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,60.168157,60.772237,50.389674,0.0
8446,strategy8447,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,60.209071,60.416198,50.121714,0.0
9223,strategy9224,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,60.072009,60.405967,50.079781,0.0
9463,strategy9464,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,60.188614,60.405967,50.376401,0.0
12323,strategy12324,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,60.202934,60.387551,50.234214,0.0
13271,strategy13272,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,60.145654,60.496000,50.013297,0.0
16439,strategy16440,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,60.274533,60.467353,50.026595,0.0
20927,strategy20928,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,60.245893,60.346627,50.091030,0.0
21214,strategy21215,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,60.360453,60.416198,50.197386,0.0
24158,strategy24159,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,60.243848,60.399828,50.107386,0.0


## Showing core testing results

In [38]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,88.888889,0.0
1,strategy2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,88.000000,0.0
2,strategy3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,88.000000,0.0
3,strategy4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,88.000000,0.0
4,strategy5,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,88.000000,0.0
...,...,...,...,...,...
38117,strategy38118,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,84.313725,0.0
38118,strategy38119,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,86.666667,0.0
38119,strategy38120,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,34.042553,0.0
38120,strategy38121,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,30.177515,0.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [39]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
267,strategy268,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,73.992674,0.0
460,strategy461,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,89.319092,0.0
870,strategy871,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,92.070485,0.0
1037,strategy1038,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,81.882353,0.0
1038,strategy1039,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,81.882353,0.0
...,...,...,...,...,...
37719,strategy37720,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((numba_indicators.relative_strength_index(pri...,81.481481,0.0
37873,strategy37874,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,82.758621,0.0
37895,strategy37896,((price_data['btc_close'][MAX_LAG:] == signals...,((numba_indicators.relative_strength_index(pri...,95.238095,0.0
37952,strategy37953,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,85.714286,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [40]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working
8446,strategy8447,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,84.530387,0.0
9223,strategy9224,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,89.221557,0.0
9463,strategy9464,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,89.808917,0.0
12323,strategy12324,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,87.586207,0.0
13271,strategy13272,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,81.781377,0.0
16439,strategy16440,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,84.393064,0.0
21214,strategy21215,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,89.473684,0.0
24165,strategy24166,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,81.938326,0.0
24535,strategy24536,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,83.425414,0.0
26244,strategy26245,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,85.714286,0.0


## Showing performance

In [41]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
0,strategy1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,27.0,232307.207653,66.373488,26174.029004,204.375907,16.0,7.844177,308.033641,195.475246,0.0,63.2375,29.9250
1,strategy2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,25.0,241570.154817,69.020044,26209.769874,146.888864,14.0,8.271616,282.477163,219.527874,0.0,65.5625,33.8875
2,strategy3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,25.0,240541.329979,68.726094,26209.769874,146.888864,14.0,8.200652,281.926934,218.592925,0.0,65.1125,34.4250
3,strategy4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,25.0,241570.154817,69.020044,26209.769874,146.888864,14.0,8.271616,282.477163,219.527874,0.0,64.7375,33.6750
4,strategy5,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,25.0,241570.154817,69.020044,26209.769874,146.888864,14.0,8.271616,282.477163,219.527874,0.0,64.4000,32.7000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38117,strategy38118,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,51.0,183640.736062,52.468782,34707.453335,241.784373,27.0,4.313586,118.068064,68.336787,0.0,59.2875,31.8000
38118,strategy38119,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,45.0,208943.733662,59.698210,38257.047010,246.970729,23.0,4.062784,148.456973,79.314878,0.0,60.0750,30.3125
38119,strategy38120,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,47.0,128927.435620,36.836410,11824.254398,248.582312,25.0,10.881872,101.460122,130.256100,0.0,55.9875,31.0500
38120,strategy38121,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,169.0,117534.415857,33.581262,11879.736982,174.601431,100.0,25.498187,30.177835,41.337584,0.0,54.0625,40.9750


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [42]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies)) & 
#     (final_perf_df['ROI (%)'] > 0) &
#     (final_perf_df['MT_PNL_D'] > 90) &
#     (final_perf_df['MT_MDD_D'] > 90)
# ]

In [43]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
267,strategy268,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,273.0,1.313272e+06,375.220709,2168.202019,134.246275,182.0,187.045746,456.911744,1013.439367,0.0,68.5250,46.3250
460,strategy461,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,749.0,1.942309e+06,554.945518,1918.809803,269.731702,213.0,1616.960451,243.514481,456.341263,0.0,64.3750,47.1000
870,strategy871,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,681.0,1.310614e+07,3744.611441,1230.760540,104.019738,383.0,5143.917006,761.150492,7479.665688,0.0,84.0375,58.8625
1037,strategy1038,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,425.0,5.995137e+05,171.289628,5333.798085,579.270524,147.0,164.295509,160.475227,119.082593,0.0,59.4250,31.6125
1038,strategy1039,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,425.0,5.995137e+05,171.289628,5333.798085,579.270524,147.0,164.295509,160.475227,119.082593,0.0,58.4625,32.4875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37719,strategy37720,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((numba_indicators.relative_strength_index(pri...,27.0,1.266723e+06,361.920838,20753.922876,130.470829,20.0,30.246010,701.052465,1142.996701,0.0,92.4500,34.6125
37873,strategy37874,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,29.0,5.815578e+05,166.159365,22373.818766,197.732626,1.0,-2.046290,639.098127,583.417161,0.0,77.6750,31.4625
37895,strategy37896,((price_data['btc_close'][MAX_LAG:] == signals...,((numba_indicators.relative_strength_index(pri...,21.0,1.094241e+06,312.640295,35549.513420,117.132205,14.0,23.484510,857.172669,1062.643815,0.0,92.3750,34.9750
37952,strategy37953,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,35.0,1.702173e+05,48.633507,26002.015763,256.097948,19.0,4.926288,168.826769,103.062682,0.0,59.0750,29.1250


### Showing performance for strategies that has passed all tests and have positive ROI

In [44]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working,MT_PNL_D,MT_MDD_D
8446,strategy8447,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,181.0,1.407325e+06,402.092877,2.417371e+03,146.538270,100.0,8.568970e+02,262.739796,1699.584100,0.0,72.3125,41.7500
9223,strategy9224,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,167.0,3.546470e+06,1013.277063,2.322274e+03,112.268475,118.0,9.744059e+02,785.658092,4611.834899,0.0,85.7625,46.0125
9463,strategy9464,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,157.0,4.218424e+06,1205.264030,6.306227e+03,155.019234,104.0,2.292113e+02,902.365359,1179.909942,0.0,89.3125,41.8250
12323,strategy12324,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,145.0,1.544229e+06,441.208291,4.875030e+03,129.185866,86.0,3.586866e+02,348.294010,934.152762,0.0,75.8500,43.7750
13271,strategy13272,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,247.0,1.558381e+06,445.251704,1.366577e+03,115.509767,144.0,9.149518e+02,329.024607,1985.250697,0.0,71.5375,47.7000
16439,strategy16440,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,173.0,1.314532e+06,375.580713,1.216668e+03,106.335551,106.0,8.165037e+02,435.510600,3351.865655,0.0,72.4125,46.9625
21214,strategy21215,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,57.0,1.311102e+06,374.600711,6.327368e+03,112.859020,38.0,2.316509e+02,592.637524,2103.088995,0.0,84.6500,43.0750
24165,strategy24166,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,227.0,9.885979e+05,282.456539,1.536575e+03,133.864393,144.0,6.804719e+02,257.737729,1345.208877,0.0,67.2625,45.2625
24535,strategy24536,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,181.0,1.319164e+06,376.903951,5.445112e+03,132.526247,112.0,3.041977e+02,288.155069,618.704041,0.0,71.7875,44.5250
26244,strategy26245,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,161.0,2.422147e+06,692.042126,6.481028e+03,115.720350,112.0,2.545482e+02,537.765555,958.849338,0.0,80.3500,44.6875


In [45]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

In [46]:
# buy_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['buy'].tolist()

# print(buy_str_list[3])

In [47]:
# sell_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['sell'].tolist()

# print(sell_str_list[3])

### Showing Monte Carlo simulation

In [48]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,42.345688,5.0,234129.359909,66.894103,1.579715,0.9859,0.0
1,strategy2,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,42.772713,5.0,238987.112790,68.282032,1.596392,0.9827,0.0
2,strategy3,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,43.054214,5.0,239376.151283,68.393186,1.588536,0.9807,0.0
3,strategy4,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,44.051234,5.0,237353.913213,67.815404,1.539467,0.9796,0.0
4,strategy5,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,41.985490,5.0,240625.978660,68.750280,1.637477,0.9803,0.0
...,...,...,...,...,...,...,...,...,...,...
38117,strategy38118,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,124.408290,16.0,187301.163888,53.514618,0.430153,0.8798,0.0
38118,strategy38119,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,104.000562,13.0,216005.341344,61.715812,0.593418,0.9207,0.0
38119,strategy38120,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,115.419830,18.0,127383.920801,36.395406,0.315331,0.8328,0.0
38120,strategy38121,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,248.547921,83.0,111058.971548,31.731135,0.127666,0.6987,0.0


### Showing simulation results for strategies that has passed entry and core testing and have positive ROI

In [49]:
mc_strategies_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
267,strategy268,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,15.412819,7.0,1.310137e+06,374.324910,24.286597,1.0000,0.0
460,strategy461,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,13.860997,13.0,1.930495e+06,551.570116,39.792961,1.0000,0.0
870,strategy871,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,0.730962,3.0,1.309565e+07,3741.615319,5118.754244,1.0000,0.0
1037,strategy1038,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,88.044414,45.0,5.956508e+05,170.185951,1.932956,1.0000,0.0
1038,strategy1039,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,87.705669,45.0,5.987433e+05,171.069520,1.950496,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...
37719,strategy37720,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((numba_indicators.relative_strength_index(pri...,10.761445,2.0,1.265425e+06,361.549980,33.596786,1.0000,0.0
37873,strategy37874,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,23.505039,3.0,5.895000e+05,168.428560,7.165636,1.0000,0.0
37895,strategy37896,((price_data['btc_close'][MAX_LAG:] == signals...,((numba_indicators.relative_strength_index(pri...,4.380715,1.0,1.078291e+06,308.083214,70.327147,1.0000,0.0
37952,strategy37953,((price_data['btc_close'][MAX_LAG:] < numba_in...,((numba_indicators.relative_strength_index(pri...,95.418960,10.0,1.721804e+05,49.194412,0.515562,0.9184,0.0


### Showing simulation results for strategies that has passed all tests and have positive ROI

In [50]:
mc_strategies_list2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
8446,strategy8447,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,10.181588,10.0,1.384047e+06,395.441961,38.838926,1.0000,0.0
9223,strategy9224,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,1.662558,3.0,3.541272e+06,1011.791950,608.575545,1.0000,0.0
9463,strategy9464,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,3.261562,3.0,4.219482e+06,1205.566150,369.628481,1.0000,0.0
12323,strategy12324,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,8.760395,5.0,1.529723e+06,437.063616,49.890855,1.0000,0.0
13271,strategy13272,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,6.492154,5.0,1.544262e+06,441.217664,67.961674,1.0000,0.0
16439,strategy16440,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,3.736622,3.0,1.303886e+06,372.538954,99.699390,1.0000,0.0
21214,strategy21215,((price_data['btc_close'][MAX_LAG:] > numba_in...,((price_data['btc_close'][MAX_LAG:] > numba_in...,4.557618,2.0,1.302347e+06,372.099093,81.643326,1.0000,0.0
24165,strategy24166,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] < get_lag(...,12.456906,7.0,9.806121e+05,280.174883,22.491530,1.0000,0.0
24535,strategy24536,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,21.216313,11.0,1.304646e+06,372.755952,17.569309,1.0000,0.0
26244,strategy26245,((price_data['btc_close'][MAX_LAG:] < numba_in...,((price_data['btc_close'][MAX_LAG:] == signals...,9.505144,4.0,2.406440e+06,687.554175,72.334954,1.0000,0.0


In [51]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_btc_experiment.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_btc_experiment.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_btc_experiment.csv', index=False)
final_perf_df.to_csv('testing_results/perf_btc_experiment.csv', index=False)
final_mc_df.to_csv('testing_results/mc_btc_experiment.csv', index=False)

# Showing equity curves

In [ ]:
# plt.plot(equity_curve_dict[19532][0]);